In [2]:
from data_loader import load_data

greedy → simulated annealing → tabu search

This solution combines three optimization techniques: Greedy, Simulated Annealing (SA), and Tabu Search. First, a quick initial solution is generated using a greedy approach, which assigns each customer to the nearest depot that can accommodate their demand. Next, Simulated Annealing is applied to iteratively improve the solution by exploring neighboring solutions with a probability of accepting worse solutions, which decreases over time to avoid local minima. Finally, Tabu Search further refines the solution by maintaining a tabu list that prevents revisiting recently explored solutions, encouraging exploration of new, potentially better solutions. Together, these techniques balance exploration and exploitation, leading to an optimized solution for the facility location problem.

In [4]:
import random
import numpy as np

def total_cost(solution, setup_costs, costs):
    depot_cost = sum(setup_costs[d] for d in solution['open_depots'])
    transport_cost = sum(costs[i][solution['assignment'][i]] for i in range(len(solution['assignment'])))
    return depot_cost + transport_cost

def delta_cost(old_solution, new_solution, setup_costs, costs):
    old_open = set(old_solution['open_depots'])
    new_open = set(new_solution['open_depots'])
    depot_diff = sum(setup_costs[d] for d in new_open - old_open) - sum(setup_costs[d] for d in old_open - new_open)
    assignment_old = old_solution['assignment']
    assignment_new = new_solution['assignment']
    transport_diff = sum(
        costs[i][assignment_new[i]] - costs[i][assignment_old[i]]
        for i in range(len(assignment_old)) if assignment_new[i] != assignment_old[i]
    )
    return depot_diff + transport_diff

def is_feasible(assignment, depot_capacities, customer_demands, n_depots):
    depot_loads = [0] * n_depots
    for i, d in enumerate(assignment):
        depot_loads[d] += customer_demands[i]
    return all(depot_loads[i] <= depot_capacities[i] for i in range(n_depots))

def initial_solution(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs):
    assignment = [-1] * n_customers
    remaining_capacity = depot_capacities[:]
    for i in range(n_customers):
        sorted_depots = sorted(range(n_depots), key=lambda d: costs[i][d])
        for d in sorted_depots:
            if remaining_capacity[d] >= customer_demands[i]:
                assignment[i] = d
                remaining_capacity[d] -= customer_demands[i]
                break
    open_depots = list(set(assignment))
    return {'assignment': assignment, 'open_depots': open_depots}

def get_neighbors_limited(solution, n_depots, depot_capacities, customer_demands, k=3):
    neighbors = []
    assignment = solution['assignment']
    for _ in range(k):
        i = random.randint(0, len(assignment)-1)
        current_d = assignment[i]
        for d in range(n_depots):
            if d != current_d:
                new_assignment = assignment[:]
                new_assignment[i] = d
                if is_feasible(new_assignment, depot_capacities, customer_demands, n_depots):
                    open_depots = list(set(new_assignment))
                    neighbors.append({'assignment': new_assignment, 'open_depots': open_depots})
    return neighbors

def simulated_annealing(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs,
                        initial_temp=1000, cooling_rate=0.95, max_iter=100, neighbor_k=5):
    greedy = initial_solution(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs)
    current = tabu_search_optimized(greedy, n_depots, depot_capacities, customer_demands, setup_costs, costs)   
    best = current
    temp = initial_temp
    for _ in range(max_iter):
        neighbors = get_neighbors_limited(current, n_depots, depot_capacities, customer_demands, k=neighbor_k)
        if not neighbors:
            break
        candidate = random.choice(neighbors)
        cost_diff = total_cost(candidate, setup_costs, costs) - total_cost(current, setup_costs, costs)
        if cost_diff < 0 or random.random() < np.exp(-cost_diff / temp):
            current = candidate
            if total_cost(current, setup_costs, costs) < total_cost(best, setup_costs, costs):
                best = current
        temp *= cooling_rate
    best_cost = total_cost(best, setup_costs, costs)
    return best, best_cost


def tabu_search_optimized(initial_sol, n_depots, depot_capacities, customer_demands,
                          setup_costs, costs, max_iter=50, tabu_tenure=5, neighbor_k=10, runs=3):
    best_global = None
    for _ in range(runs):
        current = initial_sol
        best = current
        best_cost = total_cost(best, setup_costs, costs)
        tabu_list = []
        for _ in range(max_iter):
            neighbors = get_neighbors_limited(current, n_depots, depot_capacities, customer_demands, k=neighbor_k)
            neighbors = [n for n in neighbors if n['assignment'] not in tabu_list]
            if not neighbors:
                break
            candidate = min(neighbors, key=lambda s: delta_cost(current, s, setup_costs, costs))
            cost_candidate = total_cost(candidate, setup_costs, costs)
            if cost_candidate < best_cost:
                best = candidate
                best_cost = cost_candidate
            tabu_list.append(candidate['assignment'])
            if len(tabu_list) > tabu_tenure:
                tabu_list.pop(0)
            current = candidate
        if best_global is None or total_cost(best, setup_costs, costs) < total_cost(best_global, setup_costs, costs):
            best_global = best
    return best_global


In [5]:
filename = 'Dataset/wl_25'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

sa_solution, sa_cost = simulated_annealing(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs,
                               initial_temp=1000, cooling_rate=0.9, max_iter=30, neighbor_k=3)

print("Total cost : ",sa_cost)
print("Assignment : ",sa_solution)

Total cost :  799544.275
Assignment :  {'assignment': [7, 11, 0, 24, 20, 0, 1, 5, 20, 7, 3, 10, 5, 0, 6, 7, 3, 8, 20, 6, 20, 6, 10, 0, 11, 10, 12, 10, 10, 0, 15, 10, 15, 16, 11, 11, 17, 5, 23, 24, 10, 20, 7, 6, 22, 23, 23, 6, 24, 11], 'open_depots': [0, 1, 3, 5, 6, 7, 8, 10, 11, 12, 15, 16, 17, 20, 22, 23, 24]}


In [6]:
filename = 'Dataset/wl_50'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

sa_solution, sa_cost = simulated_annealing(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs,
                               initial_temp=1000, cooling_rate=0.9, max_iter=30, neighbor_k=3)

print("Total cost : ",sa_cost)
print("Assignment : ",sa_solution)

Total cost :  925627.2625
Assignment :  {'assignment': [19, 24, 25, 48, 4, 5, 36, 12, 4, 15, 22, 22, 12, 25, 14, 15, 16, 22, 41, 19, 16, 14, 22, 23, 24, 25, 26, 22, 40, 23, 5, 22, 5, 33, 24, 16, 36, 12, 45, 48, 40, 41, 15, 14, 44, 45, 4, 19, 48, 24], 'open_depots': [4, 5, 12, 14, 15, 16, 19, 22, 23, 24, 25, 26, 33, 36, 40, 41, 44, 45, 48]}


In [ ]:
filename = 'Dataset/wl_200'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

sa_solution, sa_cost = simulated_annealing(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs,
                               initial_temp=1000, cooling_rate=0.9, max_iter=30, neighbor_k=3)

print("Total cost : ",sa_cost)
print("Assignment : ",sa_solution)

Total cost :  29819.08121
Assignment :  {'assignment': [197, 95, 84, 125, 24, 53, 185, 93, 113, 198, 178, 198, 111, 51, 162, 47, 71, 23, 148, 24, 158, 137, 71, 126, 32, 48, 163, 95, 106, 34, 184, 178, 48, 93, 147, 102, 150, 20, 70, 117, 79, 108, 160, 147, 158, 84, 19, 63, 133, 126, 165, 125, 158, 32, 162, 129, 13, 106, 51, 53, 95, 162, 102, 53, 85, 171, 165, 121, 70, 38, 166, 49, 45, 64, 147, 25, 144, 5, 20, 81, 198, 4, 111, 178, 53, 160, 109, 24, 150, 99, 45, 24, 75, 184, 128, 102, 169, 176, 12, 121, 34, 165, 38, 168, 53, 25, 129, 13, 198, 23, 93, 49, 47, 95, 19, 190, 113, 163, 166, 66, 196, 52, 61, 0, 24, 126, 85, 28, 34, 107, 13, 84, 153, 5, 169, 163, 75, 70, 184, 108, 168, 106, 21, 176, 149, 45, 0, 110, 71, 70, 12, 113, 128, 0, 128, 111, 153, 12, 186, 3, 190, 71, 153, 86, 133, 86, 12, 198, 110, 195, 66, 109, 144, 70, 86, 127, 149, 176, 176, 21, 28, 66, 196, 3, 149, 178, 139, 198, 185, 79, 4, 117, 144, 19, 139, 107, 107, 137, 22, 13], 'open_depots': [0, 3, 4, 5, 12, 13, 19, 20, 21, 

In [8]:
filename = 'Dataset/wl_300'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

sa_solution, sa_cost = simulated_annealing(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs,
                               initial_temp=1000, cooling_rate=0.9, max_iter=30, neighbor_k=3)

print("Total cost : ",sa_cost)
print("Assignment : ",sa_solution)

Total cost :  73683.82339
Assignment :  {'assignment': [134, 53, 292, 256, 125, 90, 225, 275, 181, 100, 221, 275, 240, 53, 156, 202, 161, 4, 200, 118, 238, 268, 42, 226, 103, 169, 149, 147, 268, 126, 220, 51, 236, 172, 237, 48, 19, 126, 284, 212, 22, 131, 289, 12, 219, 256, 284, 48, 83, 11, 93, 71, 273, 281, 187, 175, 148, 264, 59, 166, 296, 122, 28, 34, 181, 237, 92, 126, 178, 219, 140, 253, 62, 204, 172, 148, 42, 56, 234, 58, 125, 297, 22, 219, 12, 252, 268, 140, 96, 12, 13, 27, 217, 46, 296, 28, 199, 96, 84, 266, 141, 100, 166, 219, 181, 164, 69, 13, 282, 19, 266, 170, 133, 59, 3, 201, 82, 18, 211, 84, 93, 163, 23, 175, 240, 296, 296, 62, 192, 103, 136, 264, 221, 113, 297, 164, 292, 187, 281, 2, 27, 278, 42, 20, 187, 202, 19, 122, 134, 117, 46, 24, 213, 209, 93, 186, 13, 206, 103, 257, 210, 172, 135, 19, 119, 11, 27, 232, 118, 289, 253, 103, 54, 12, 268, 284, 164, 172, 140, 155, 0, 204, 219, 264, 288, 97, 188, 138, 2, 113, 52, 238, 268, 61, 210, 46, 54, 288, 22, 0, 172, 96, 242, 38,

In [9]:
filename = 'Dataset/wl_500'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

sa_solution, sa_cost = simulated_annealing(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs,
                               initial_temp=1000, cooling_rate=0.9, max_iter=30, neighbor_k=3)

print("Total cost : ",sa_cost)
print("Assignment : ",sa_solution)

Total cost :  81305.15052000001
Assignment :  {'assignment': [432, 79, 473, 419, 269, 355, 87, 51, 60, 324, 260, 38, 31, 66, 176, 344, 161, 168, 471, 276, 94, 458, 499, 154, 174, 290, 76, 459, 437, 458, 96, 138, 475, 372, 196, 442, 141, 98, 269, 164, 427, 243, 375, 462, 369, 31, 149, 238, 245, 484, 35, 140, 256, 467, 166, 338, 384, 385, 276, 94, 420, 121, 446, 221, 379, 285, 351, 381, 133, 285, 460, 472, 184, 304, 115, 487, 406, 261, 351, 429, 94, 243, 54, 135, 217, 121, 350, 337, 470, 128, 320, 445, 272, 234, 176, 176, 403, 111, 2, 443, 268, 84, 144, 426, 109, 497, 171, 245, 183, 183, 409, 473, 8, 412, 110, 123, 356, 407, 63, 15, 443, 408, 391, 58, 214, 144, 121, 123, 337, 102, 265, 435, 386, 419, 499, 279, 225, 17, 397, 118, 336, 25, 8, 416, 64, 484, 200, 33, 433, 164, 367, 232, 325, 216, 301, 367, 111, 109, 371, 324, 256, 82, 131, 285, 135, 19, 2, 275, 367, 0, 475, 141, 407, 165, 60, 497, 76, 103, 130, 106, 443, 174, 266, 365, 252, 102, 27, 51, 489, 350, 435, 265, 333, 13, 386, 337,